[View in Colaboratory](https://colab.research.google.com/github/RaulMedeiros/Deep_Transfer_Learning_Framework/blob/master/Deep_Transfer_Learning_Framework.ipynb)

In [20]:
# # Remove Older Files.
# !ls
# !rm -r *
# !ls

adc.json  data	data.zip  out


**CONFIG DEEP FEATURE EXTRACTOR**

In [0]:
src_drive_folder_id = '1A5ByNWx7Nr72uOzXMy3bwLHDwShQgqYc'
drive_filenames = ['data.zip']
out_drive_folder_id = '1Zkx5UfATBDCmQP4A4VDzFYTjjy6LIbq3'

#Especifique a topologia a ser utilizada.
model_name_list = ['MobileNet',
                   'ResNet50',
                   'Xception',
                   'VGG16',
                   'VGG19',
                   'InceptionV3',
                   'InceptionResNetV2',
                   'NASNetMobile',
                   'NASNetLarge','DenseNet121','DenseNet169','DenseNet201']  

# #Especifique a topologia a ser utilizada.
model_name_list = ['DenseNet201']  

#Especifique o formato de saida
output_fmt = ['txt','npy']    

**Authenticate and create the PyDrive client**

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Donwload and Unzip Data.zip**

In [23]:
#!pip3 install -q pandas
import pandas as pd

import zipfile

def unzip(path_to_zip_file,extract_to='./'):
  zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
  zip_ref.extractall(extract_to)
  zip_ref.close()
  return True

def load_and_unzip_file(folder_id,target_filenames):
  """Download and unzip dataset"""   

  info = pd.DataFrame(drive.ListFile({'q': "'"+folder_id +
                                   "' in parents and trashed=false"}).GetList())  
  if (info.empty):
    print('Empty Drive Folder!')
    return False
  
  else:
    for f_name, drive_id in zip(info['title'].values, info['id'].values):
      if (f_name in target_filenames):      
        print(drive_id,f_name)
        downloaded = drive.CreateFile({'id': drive_id }).GetContentFile(f_name)
        unzip(f_name, extract_to='./')
        !ls
  return True

## Load and Unconpress data
load_and_unzip_file(src_drive_folder_id,drive_filenames)

1meShW1ucL2OlRqPCYN5eLCEvgmxSlchX data.zip
adc.json  data	data.zip


True

**Sample of files into class/folder 0**

In [24]:
!ls data/0

100.jpg  142.jpg  184.jpg  225.jpg  267.jpg  308.jpg  34.jpg   391.jpg	63.jpg
101.jpg  143.jpg  185.jpg  226.jpg  268.jpg  309.jpg  350.jpg  392.jpg	64.jpg
102.jpg  144.jpg  186.jpg  227.jpg  269.jpg  30.jpg   351.jpg  393.jpg	65.jpg
103.jpg  145.jpg  187.jpg  228.jpg  26.jpg   310.jpg  352.jpg  394.jpg	66.jpg
104.jpg  146.jpg  188.jpg  229.jpg  270.jpg  311.jpg  353.jpg  395.jpg	67.jpg
105.jpg  147.jpg  189.jpg  22.jpg   271.jpg  312.jpg  354.jpg  396.jpg	68.jpg
106.jpg  148.jpg  18.jpg   230.jpg  272.jpg  313.jpg  355.jpg  397.jpg	69.jpg
107.jpg  149.jpg  190.jpg  231.jpg  273.jpg  314.jpg  356.jpg  398.jpg	6.jpg
108.jpg  14.jpg   191.jpg  232.jpg  274.jpg  315.jpg  357.jpg  399.jpg	70.jpg
109.jpg  150.jpg  192.jpg  233.jpg  275.jpg  316.jpg  358.jpg  39.jpg	71.jpg
10.jpg	 151.jpg  193.jpg  234.jpg  276.jpg  317.jpg  359.jpg  3.jpg	72.jpg
110.jpg  152.jpg  194.jpg  235.jpg  277.jpg  318.jpg  35.jpg   400.jpg	73.jpg
111.jpg  153.jpg  195.jpg  236.jpg  278.jpg  319.jpg  360.jpg  401.jp

**Install Libraries **

In [0]:
# https://keras.io/
!pip install -q keras
import keras

# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

from IPython.display import clear_output

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import keras
from keras import backend as tf
from scipy import misc

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetMobile
from keras.applications.nasnet import NASNetLarge
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201

from keras.applications.inception_resnet_v2 import preprocess_input as ppi_inceptionresnet_v2
from keras.applications.vgg16 import preprocess_input as ppi_vgg16
from keras.applications.vgg19 import preprocess_input as ppi_vgg19
from keras.applications.xception import preprocess_input as ppi_xception
from keras.applications.resnet50 import preprocess_input as ppi_resnet50
from keras.applications.inception_v3 import preprocess_input as ppi_inception_v3
from keras.applications.mobilenet import preprocess_input as ppi_mobilenet
from keras.applications.nasnet import preprocess_input as ppi_nasnet
from keras.applications.densenet import preprocess_input as ppi_densenet


from keras.preprocessing import image
from keras.layers import Input

np.set_printoptions(threshold=np.nan)
keras.backend.backend()

import time
current_milli_time = lambda: int(round(time.time() * 1000))

In [0]:
def build_model(model_name,pooltype='max'):
    if ('VGG16' is model_name):
        model = VGG16(weights='imagenet',pooling=pooltype, include_top=False)    
    if ('VGG19' is model_name):
        model = VGG19(weights='imagenet',pooling=pooltype, include_top=False)
    if ('MobileNet' is model_name):
        input_tensor = Input(shape=(224,224,3))
        model = MobileNet(weights='imagenet',pooling=pooltype,input_shape=(224,224,3),include_top=False)  
    if ('ResNet50' is model_name):
        model = ResNet50(weights='imagenet',pooling=pooltype, include_top=False)       
    if ('InceptionV3' is model_name):
        model = InceptionV3(weights='imagenet',pooling=pooltype, include_top=False)
    if ('Xception' is model_name):
        model = Xception(weights='imagenet',pooling=pooltype, include_top=False)
    if ('InceptionResNetV2' is model_name):
        model = InceptionResNetV2(weights='imagenet',pooling=pooltype, include_top=False)
    if ('NASNetMobile' is model_name):
        model = NASNetMobile(weights='imagenet',pooling=pooltype, include_top=False)
    if ('NASNetLarge' is model_name):
        model = NASNetLarge(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet121' is model_name):
        model = DenseNet121(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet169' is model_name):
        model = DenseNet169(weights='imagenet',pooling=pooltype, include_top=False)
    if ('DenseNet201' is model_name):
        model = DenseNet201(weights='imagenet',pooling=pooltype, include_top=False)    
                
    if (model_name in ['InceptionV3','Xception','InceptionResNetV2']):
        targetSize = 299  
    elif(model_name in ['NASNetLarge']):
        targetSize = 331
    else:    
        targetSize = 224   
    
    return model,targetSize
  
def extractDeepFeatures(file_id,model,model_name,target_size,log=False):
    
    x = load_img(file_id,target_size)
    x = image.img_to_array(x)    
    x = np.expand_dims(x, axis=0)

    if (model_name is 'VGG16'):
        x = ppi_vgg16(x)        
    if (model_name is 'VGG19'):
        x = ppi_vgg19(x)
    if (model_name is 'ResNet50'):
        x = ppi_resnet50(x)
    if (model_name is 'MobileNet'):
        x = ppi_mobilenet(x)
    if (model_name is 'Xception'):
        x = ppi_xception(x) 
    if (model_name is 'InceptionV3'):
        x = ppi_inception_v3(x)
    if (model_name is 'InceptionResNetV2'):
        x = ppi_inceptionresnet_v2(x)
    if (model_name is 'NASNetMobile'):
        x = ppi_nasnet(x)
    if (model_name is 'NASNetLarge'):
        x = ppi_nasnet(x)
    if (model_name is 'DenseNet121'):
        x = ppi_densenet(x)
    if (model_name is 'DenseNet169'):
        x = ppi_densenet(x)
    if (model_name is 'DenseNet201'):
        x = ppi_densenet(x)

    timeStart = current_milli_time()
    features = model.predict(x).reshape(-1)
    process_time = current_milli_time()-timeStart    
    if(log): print('Load process: Complete',process_time)

    return features,process_time

def load_img(filepath,target_size):
    """Load dataset"""
    #LOAD image to be deep extracted         
    if (filepath.endswith('jpg') or filepath.endswith('JPG') ):
        img = cv2.imread((filepath))
    elif (filepath.endswith('txt')):
        img = np.loadtxt(filepath).copy()
    elif (filepath.endswith('npy')):
        img = np.load(filepath).copy()
    elif (filepath.endswith('png') or filepath.endswith('PNG')):
        img = misc.imread(filepath).copy()
    img = cv2.resize(img, (target_size,target_size), interpolation = cv2.INTER_NEAREST)
    return img

**Iterate through each model and each image to extract deep features **

In [0]:
def deep_extractor(model_name,numOfClasses,out_folder_id,output_fmt=['npy'],
              srcPath='./data',outPath='./out'):
  print(model_name)
  cont = 0

  # For each class    
  model,targetSize = build_model(model_name)
  data = []
  process_time_list= []

  # Start chronometer
  timeStart = current_milli_time()

  for fileIdx in range(numOfClasses):
    folderPath = srcPath+'/'+str(fileIdx)

    # For each sample
    for subdir, dirs, files in os.walk(folderPath):
      for idx,name in enumerate(files):
        filePath = subdir + os.sep + name
        cont=cont+1

        clear_output()             
        print(cont,filePath,model_name) 

        features,process_time = extractDeepFeatures(filePath,model,model_name,targetSize)
        process_time_list.append(process_time)
        
        features = np.hstack((features,fileIdx))
        data.append(features)

  # Compute extraction time      
  process_time = current_milli_time()-timeStart    
    
  # Creates a folder if don't exist
  os.makedirs(outPath+'/', exist_ok=True) 
  # Saves a txt file w/ deep features
  if('txt' in output_fmt):
    np.savetxt(outPath+'/'+model_name+'.txt',data,fmt="%.8f")
    #Upload txt file to folder (by id)
    file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                          "id": out_folder_id}]})
    file.SetContentFile(outPath+'/'+model_name+'.txt')
    file.Upload()

  #Saves a npy file w/ deep features
  if('npy' in output_fmt):
    np.save(outPath+'/'+model_name+'.npy',data)      
    #Upload npy file to folder (by id)
    file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                          "id": out_folder_id}]})
    file.SetContentFile(outPath+'/'+model_name+'.npy')
    file.Upload() 

  #Save process time into a file
  np.savetxt(outPath+'/'+model_name+'_time.txt',process_time_list,fmt="%d")
  #Upload txt file to folder (by id)
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink",
                                        "id": out_folder_id}]})
  file.SetContentFile(outPath+'/'+model_name+'_time.txt')
  file.Upload()
  return process_time

In [28]:
%%time

## Especifique o caminho da pasta onde estão as imagens a serem processadas.
srcPath = './data'

## Especifique o caminho da pasta onde serão armazenadas as caracteristicas extridas.
outPath = './out'                         

#Encontra o numero de classes do problema.
numOfClasses = len(glob.glob('./'+srcPath+'/*'))

# For each deep model    
for model_name in model_name_list:
  process_time = deep_extractor(model_name,numOfClasses,out_drive_folder_id,output_fmt,srcPath,outPath)
  print('Load process: Complete',process_time)
  

2070 ./data/3/080.jpg DenseNet201
CPU times: user 4min 53s, sys: 43.7 s, total: 5min 36s
Wall time: 5min 27s


In [29]:
print('Done')

Done


## Qual o tempo de estação médio de cada um dos modelos?

